<a href="https://colab.research.google.com/github/ab17254/ce802/blob/main/CE802_P2_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import tree, metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, make_scorer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

sns.set_style('whitegrid')

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving CE802_P2_Data.csv to CE802_P2_Data.csv
Saving CE802_P2_Test.csv to CE802_P2_Test.csv
User uploaded file "CE802_P2_Data.csv" with length 119990 bytes
User uploaded file "CE802_P2_Test.csv" with length 113062 bytes


In [ ]:
data = pd.read_csv('/content/CE802_P2_Data.csv')
test = pd.read_csv('/content/CE802_P2_Test.csv')


In [92]:
data.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,11.7,4.02,-4.34,9.90,29.79,89.58,0.63,23,10.35,158.56,-7.88,0.03,1,1.96,NaN,False
1,11.7,4.20,-3.68,10.98,17.46,179.58,0.05,11,8.30,110.56,-3.10,0.84,1,1.50,NaN,False
2,37.7,25.80,3.60,0.48,12.24,407.58,-0.29,230,4.06,254.56,6.68,21.60,10,7.63,NaN,True
3,7.7,5.40,0.30,9.42,19.86,119.58,0.29,12,7.61,66.56,-1.84,1.05,1,2.27,12.17,True
4,15.7,5.58,-2.58,16.34,17.49,146.58,-0.64,25,9.86,106.56,-4.36,1.68,1,1.28,NaN,False


In [91]:
test.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,37.7,33.15,9.26,3.88,5.91,-12.42,-0.77,165,3.32,334.56,4.42,18.75,10,7.78,NaN,NaN
1,17.7,24.30,6.58,3.40,10.47,-57.42,-0.77,20,3.22,224.56,0.28,15.30,10,8.13,NaN,NaN
2,37.7,17.55,10.16,-1.94,6.90,92.58,-0.84,215,4.71,94.56,6.98,17.70,10,10.08,13.27,NaN
3,11.7,2.61,-8.28,8.96,23.22,161.58,1.69,6,8.54,64.56,-6.52,1.32,1,1.80,11.73,NaN
4,7.7,5.55,1.52,8.50,21.66,95.58,2.41,22,10.74,128.56,-3.84,2.76,1,2.22,12.31,NaN


In [5]:
data.dtypes

F1       float64
F2       float64
F3       float64
F4       float64
F5       float64
F6       float64
F7       float64
F8         int64
F9       float64
F10      float64
F11      float64
F12      float64
F13        int64
F14      float64
F15      float64
Class       bool
dtype: object

In [6]:
cleanup = {'Class': {False: 0, True:1}}
data.replace(cleanup, inplace=True)
data

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,Class
0,11.7,4.02,-4.34,9.90,29.79,89.58,0.63,23,10.35,158.56,-7.88,0.03,1,1.96,NaN,0
1,11.7,4.20,-3.68,10.98,17.46,179.58,0.05,11,8.30,110.56,-3.10,0.84,1,1.50,NaN,0
2,37.7,25.80,3.60,0.48,12.24,407.58,-0.29,230,4.06,254.56,6.68,21.60,10,7.63,NaN,1
3,7.7,5.40,0.30,9.42,19.86,119.58,0.29,12,7.61,66.56,-1.84,1.05,1,2.27,12.17,1
4,15.7,5.58,-2.58,16.34,17.49,146.58,-0.64,25,9.86,106.56,-4.36,1.68,1,1.28,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,37.7,33.90,5.80,6.62,10.71,362.58,-1.52,165,5.52,444.56,-1.96,15.30,10,6.93,8.76,0
1496,17.7,29.40,8.00,-0.48,3.54,-102.42,1.17,100,3.76,304.56,6.78,29.25,10,7.53,12.19,1
1497,11.7,2.13,-0.92,12.12,22.65,95.58,-0.57,10,8.47,76.56,-4.76,2.34,1,1.89,NaN,1
1498,11.7,2.94,0.64,11.68,17.49,146.58,1.47,20,8.57,116.56,-5.00,2.67,1,1.48,11.55,0


In [7]:
data['F15'].fillna(data['F15'].mean(), inplace=True)

In [8]:
features = {'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15'}

In [9]:
x = data.loc[:, features].to_numpy()
y = data.loc[:, 'Class'].to_numpy()
print(x.shape, y.shape)

(1500, 15) (1500,)


In [113]:
x_test_data = test.loc[:, features].to_numpy()
print(x.shape)

(1500, 15)


In [85]:
#preprocessing of training data
k_fold = KFold(n_splits=5)
x_copy = x.copy()
for train_indices, test_indices in k_fold.split(x):
    mean = np.nanmean(x_copy[train_indices, 1])
    x[np.isnan(x_copy[:,1]), 1] = mean
    ss = StandardScaler()
    ss.fit(x[train_indices, :])
    x_train = ss.transform(x[train_indices, :])
    x_test = ss.transform(x[test_indices, :])
    y_train = y[train_indices]
    y_test = y[test_indices]


In [116]:
# preprocessing of test data
mean = np.nanmean(x_test_data)
x_test_data[np.isnan(x_test_data)] = mean
ss = StandardScaler()
x_test_fitted = ss.fit_transform(x_test_data)

array([[ 0.99831561,  0.47974147,  0.68507474, ...,  1.38202115,
        -1.16612175,  0.91915283],
       [ 0.99831561, -0.76910679,  0.35049551, ...,  0.69669145,
        -1.2059563 ,  0.11789639],
       [-0.8054123 ,  0.9103788 ,  0.58324628, ...,  0.17398236,
        -0.61242159, -0.82904305],
       ...,
       [-0.9227573 , -0.44182242, -0.93254311, ..., -1.15021399,
         1.39125597, -0.69792836],
       [ 0.99831561, -0.78633228, -0.87435542, ..., -1.05031848,
         0.4631111 , -0.85817964],
       [-1.07813827,  0.26442281,  1.1651232 , ...,  2.38097629,
        -0.01091998,  0.48210386]])

In [118]:
print(x_train_test.shape, x_test_test.shape)
print(x_test_fitted.shape)

(1200, 15) (1200, 15)
(1500, 15)


## a) Investigate the performance of a number of machine learning procedures on this dataset.

### Decision Tree classifier

In [35]:
model_dt = DecisionTreeClassifier()
model_dt.fit(x_train, y_train)
dt_pred = model_dt.predict(x_test)
print(confusion_matrix(y_test, dt_pred), '\n', classification_report(y_test, dt_pred))

[[133  38]
 [ 31  98]] 
               precision    recall  f1-score   support

           0       0.81      0.78      0.79       171
           1       0.72      0.76      0.74       129

    accuracy                           0.77       300
   macro avg       0.77      0.77      0.77       300
weighted avg       0.77      0.77      0.77       300



#### Tuned Decision Tree classifier

In [64]:
dt_param_grid = (dict(min_samples_split=list(range(2,51))))
dt_cv = GridSearchCV(model_dt, dt_param_grid, refit=True)
# dt_cv.get_params()
dt_cv.fit(x_train, y_train)
print(dt_cv.best_params_, dt_cv.best_score_)
dt_cv_pred = dt_cv.predict(x_test)
print(confusion_matrix(y_test, dt_cv_pred), '\n', classification_report(y_test, dt_cv_pred))

{'min_samples_split': 35} 0.7883333333333333
[[143  28]
 [ 30  99]] 
               precision    recall  f1-score   support

           0       0.83      0.84      0.83       171
           1       0.78      0.77      0.77       129

    accuracy                           0.81       300
   macro avg       0.80      0.80      0.80       300
weighted avg       0.81      0.81      0.81       300



### KNN classifier

In [36]:
model_knn = KNeighborsClassifier()
model_knn.fit(x_train, y_train)
knn_pred = model_knn.predict(x_test)
print(confusion_matrix(y_test, knn_pred), '\n', classification_report(y_test, knn_pred))

[[139  32]
 [ 47  82]] 
               precision    recall  f1-score   support

           0       0.75      0.81      0.78       171
           1       0.72      0.64      0.67       129

    accuracy                           0.74       300
   macro avg       0.73      0.72      0.73       300
weighted avg       0.74      0.74      0.73       300



#### Tuned KNN classifier

In [52]:
knn_param_grid = dict(n_neighbors=list(range(1,31)))
knn_cv = GridSearchCV(model_knn, knn_param_grid, refit=True)
# knn_cv.get_params()
knn_cv.fit(x_train, y_train)
print(knn_cv.best_params_, knn_cv.best_score_)
knn_cv_pred = knn_cv.predict(x_test)
print(confusion_matrix(y_test, knn_cv_pred), '\n', classification_report(y_test, knn_cv_pred))

{'n_neighbors': 29} 0.7666666666666666
[[145  26]
 [ 33  96]] 
               precision    recall  f1-score   support

           0       0.81      0.85      0.83       171
           1       0.79      0.74      0.76       129

    accuracy                           0.80       300
   macro avg       0.80      0.80      0.80       300
weighted avg       0.80      0.80      0.80       300



### Support Vector classifier

In [37]:
model_svc = SVC()
model_svc.fit(x_train, y_train)
svc_pred = model_svc.predict(x_test)
print(confusion_matrix(y_test, svc_pred), '\n', classification_report(y_test, svc_pred))

[[137  34]
 [ 20 109]] 
               precision    recall  f1-score   support

           0       0.87      0.80      0.84       171
           1       0.76      0.84      0.80       129

    accuracy                           0.82       300
   macro avg       0.82      0.82      0.82       300
weighted avg       0.83      0.82      0.82       300



#### Tuned Support Vector classifier

In [53]:
svc_param_grid = {'C':[1,10,100,1000],
              'gamma':[1,0.1,0.001,0.0001],
              'kernel': ['rbf']}
svc_cv = GridSearchCV(SVC(), svc_param_grid, refit=True)
# svc_cv.get_params()
svc_cv.fit(x_train, y_train)
print(svc_cv.best_params_, svc_cv.best_score_)
svc_cv_pred = svc_cv.predict(x_test)
print(confusion_matrix(y_test, svc_cv_pred), '\n', classification_report(y_test, svc_cv_pred))

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'} 0.8675
[[152  19]
 [ 18 111]] 
               precision    recall  f1-score   support

           0       0.89      0.89      0.89       171
           1       0.85      0.86      0.86       129

    accuracy                           0.88       300
   macro avg       0.87      0.87      0.87       300
weighted avg       0.88      0.88      0.88       300



## b) Prediction on a hold-out test set

### Tuned Support Vector classifier

In [128]:
y_test = svc_cv.predict(x_test_fitted)
test['Class'] = y_test

In [129]:
test.groupby('Class').count()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15
Class,,,,,,,,,,,,,,,
0,840,840,840,840,840,840,840,840,840,840,840,840,840,840,607
1,660,660,660,660,660,660,660,660,660,660,660,660,660,660,143


In [132]:
test.to_csv('CE802_P2_Test_Ouput.csv') #Change file name